In [1]:
import json
import numpy
import operator

In [2]:
pool_depth = 100

In [3]:
id_pos = 1
score_pos = 2
ans_length = 3 
id_length = 64

clus_type = 'Cluster Aggregate'

In [4]:
def parse_sub(sub, index_of_ad, len_answ, len_id):
    agg = []
    entries = []
    for entry in sub:
        if type(entry) != list:
            agg.append(entry)
        elif len(entry) != len_answ:
            agg.append(entry)
        elif len(entry[index_of_ad]) != id_length:
            agg.append(entry)
        else:
            entries.append(entry)
        
    return [agg, entries]

In [5]:
def AvgPrecision(gt, sub, index_of_ad, index_of_score):
    score = 100
    seen = []
    rel_docs = len(gt)
    num_hits = 0
    num_retrieved = 0
    cumpre = 0
    for entry in sub:
        num_retrieved += 1
        ad_id = entry[index_of_ad]
        # excessive checking of dupes
        if ad_id in seen:
            print "DUPLICATE"
            return None
        seen.append(ad_id)
        # excessive checking of order
        if entry[index_of_score] > score:
            print "ORDER PROBLEM"
            return None
        score = entry[index_of_score]
        if ad_id in gt:
            num_hits += 1
            cumpre += num_hits/float(num_retrieved)
            
    return cumpre/rel_docs

In [6]:
def AggMetric(gt_answ, sub_answ, feature):
    function = agg_metric_params[feature]['function']
    amount = agg_metric_params[feature]['amount']
    diff = gt_answ - sub_answ
    if function == 'multiply':
        if numpy.absolute(diff) <= (gt_answ * amount):
            metric = 1
        else:
            metric = 0
    elif function == 'add':
        if numpy.absolute(diff) <= amount:
            metric = 1
        else:
            metric = 0
        
    return metric

In [7]:
def clean_sub(sub, index_of_ad, index_of_score, expected_length):
    seen = []
    uniqs = []
    # explicitly check ordering
    i = 100
    sub.sort(key=operator.itemgetter(index_of_score), reverse=True)
    for entry in sub:
        if entry[index_of_score] > i:
            print "ORDERING PROBLEM"
            return None
        i = entry[index_of_score]
        #else:
        #    print type(entry)
    # de-dupe
    for entry in sub:
        if entry[index_of_ad] not in seen:
            uniqs.append(entry)
            seen.append(entry[index_of_ad])
    # explicitly sort by score after de-duping
    uniqs.sort(key=operator.itemgetter(index_of_score), reverse=True)
    # explicitly check ordering
    i = 100
    for entry in uniqs:
        if entry[index_of_score] > i:
            print "ORDERING PROBLEM"
            return None
        i = entry[index_of_score]
    return uniqs

In [8]:
dd_agg_answer_key_file =  '../../../data/aggregate_dd/saved_dictionaries/dd_agg_answer_key.json'

with open(dd_agg_answer_key_file, 'r') as f:
    answer_key = eval(f.read()) 
    
agg_metric_file = '../../../data/aggregate_dd/agg_metric_parameters.json'

with open(agg_metric_file, 'r') as f:
    agg_metric_params = eval(f.read()) 

### To avoid confusing dependecies, we'll get submission data from raw submissions

In [9]:
submission_path = '../../../data/team_submissions/'

In [10]:
answers = {}

# NYU

In [11]:
answers['NYU'] = {}
answers['NYU'][clus_type] = {}

In [12]:
file_path = submission_path + 'Georgetown/DomainDiscovery/NYU_aggregate.json'
f = open(file_path, 'r')
data = eval(f.read())

for qid in answer_key.keys():
    submissions = []
    agg_answer = []
    for entry in data:
        # We are only evaluating some of them
        if entry['id'] == qid:
            # NEW
            parsed_submission = parse_sub(entry['answer'], id_pos, ans_length, id_length)
            agg_answer = parsed_submission[0]
            submissions = clean_sub(parsed_submission[1], id_pos, score_pos, ans_length)
    # Reconfirming no dupes
    if len(submissions) != len(list(set(tuple(i) for i in submissions))):
        print "PROBLEM WITH DUPLICATED DOC IDS"
        break
    # Now that submissions are sorted, only take top N
    if len(submissions) > pool_depth:
        submissions = submissions[:pool_depth]
        print "POOL DEPTH REACHED"
    # NEW
    answers['NYU'][clus_type][qid] = {}
    answers['NYU'][clus_type][qid]['agg_answer'] = agg_answer
    answers['NYU'][clus_type][qid]['sub'] = submissions

# HG

In [13]:
answers['HG'] = {}
answers['HG'][clus_type] = {}

In [14]:
file_path = submission_path + 'Georgetown/DomainDiscovery/Georgetown_Submission_2_8/HG/HG_post_aggregate.json'
f = open(file_path, 'r')
data = eval(f.read())

for qid in answer_key.keys():
    submissions = []
    agg_answer = []
    for entry in data:
        # We are only evaluating some of them
        if entry['id'] == qid:
            parsed_submission = parse_sub(entry['answer'], id_pos, ans_length, id_length)
            agg_answer = parsed_submission[0]
            submissions = clean_sub(parsed_submission[1], id_pos, score_pos, ans_length)
    # Reconfirming no dupes
    if len(submissions) != len(list(set(tuple(i) for i in submissions))):
        print "PROBLEM WITH DUPLICATED DOC IDS"
        break
    # Now that submissions are sorted, only take top N
    if len(submissions) > pool_depth:
        submissions = submissions[:pool_depth]
        print "POOL DEPTH REACHED"
    answers['HG'][clus_type][qid] = {}
    answers['HG'][clus_type][qid]['agg_answer'] = agg_answer
    answers['HG'][clus_type][qid]['sub'] = submissions

POOL DEPTH REACHED
POOL DEPTH REACHED
POOL DEPTH REACHED
POOL DEPTH REACHED
POOL DEPTH REACHED


# JPL

In [15]:
answers['JPL'] = {}
answers['JPL'][clus_type] = {}

In [16]:
# Georgetown's JPL submission did not match format
id_pos = 0
score_pos = 1
ans_length = 2 
id_length = 64

In [17]:
file_path = submission_path + 'Georgetown/DomainDiscovery/Georgetown_Submission_2_8/JPL/JPL_post_aggregate.json'
f = open(file_path, 'r')
data = eval(f.read())

for qid in answer_key.keys():
    submissions = []
    agg_answer = []
    for entry in data:
        # We are only evaluating some of them
        if entry['id'] == qid:
            parsed_submission = parse_sub(entry['answer'], id_pos, ans_length, id_length)
            agg_answer = parsed_submission[0]
            submissions = clean_sub(parsed_submission[1], id_pos, score_pos, ans_length)
    # Reconfirming no dupes
    if len(submissions) != len(list(set(tuple(i) for i in submissions))):
        print "PROBLEM WITH DUPLICATED DOC IDS"
        break
    # Now that submissions are sorted, only take top N
    if len(submissions) > pool_depth:
        submissions = submissions[:pool_depth]
        print "POOL DEPTH REACHED"
    answers['JPL'][clus_type][qid] = {}
    answers['JPL'][clus_type][qid]['agg_answer'] = agg_answer
    answers['JPL'][clus_type][qid]['sub'] = submissions

# Calculate Results

In [18]:
results = {}
results[clus_type] = {}

for dataset in answers.keys():
    # Georgetown's JPL submission did not conform to format
    if dataset == 'JPL':
        id_pos = 0
        score_pos = 1
    else:
        id_pos = 1
        score_pos = 2
    results[clus_type][dataset] = {}
    for quest in answers[dataset][clus_type].keys():
        results[clus_type][dataset][quest] = {}
        submission = answers[dataset][clus_type][quest]['sub']
        # NEW
        relevant_ads = answer_key[quest][dataset]['relevant_ads']
        results[clus_type][dataset][quest] = {}
        if len(relevant_ads) != 0:
            results[clus_type][dataset][quest]['avg_precision'] = AvgPrecision(relevant_ads, submission, id_pos, score_pos)
        else:
            results[clus_type][dataset][quest]['avg_precision'] = 'NA'
        
        # Agg Metric
        if answer_key[quest][dataset]['agg_answer'] == 'No relevant extractions found':
            results[clus_type][dataset][quest]['agg_metric'] = 'NA'
        elif len(answers[dataset][clus_type][qid]['agg_answer']) == 0:
            results[clus_type][dataset][quest]['agg_metric'] = 0
        else:
            metric = AggMetric(answer_key[quest][dataset]['agg_answer'],
                               answers[dataset][clus_type][qid]['agg_answer'][0],
                               answer_key[quest]['feature'])
            results[clus_type][dataset][quest]['agg_metric'] = metric

In [19]:
# Confirm all 0 scores had relevant docs in GT
for dataset in results['Cluster Aggregate'].keys():
    for qid in results['Cluster Aggregate'][dataset].keys():
        if results['Cluster Aggregate'][dataset][qid]['avg_precision'] == 0:
            if len(answer_key[qid][dataset]['relevant_ads']) == 0:
                print 'Found 0 avg_precision and no relevant ads for question {0}!'.format(qid)
        if results['Cluster Aggregate'][dataset][qid]['agg_metric'] == 0:
            if answer_key[qid][dataset]['agg_answer'] == 'No relevant extractions found':
                print 'Found 0 agg_metric and no relevant extractions for question {0}!'.format(qid)

## Save Results

In [20]:
output_file = 'Georgetown_DD_Cluster_Agg_Results_REWRITE.json'
with open(output_file, 'w') as outfile:
    json.dump(results, outfile, indent=2)